In [1]:
from dash import Dash, html, dcc
import plotly.express as px
import pandas as pd

import notebook_importing

import nbpackage.dashapp as ds

importing Jupyter notebook from %s % path


In [2]:
from jupyter_dash import JupyterDash

In [3]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

In [4]:
app.layout = ds.layout

In [5]:
if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
